<a href="https://colab.research.google.com/github/ekaratnida/Data_Streaming_and_Realtime_Analytics/blob/main/Week6_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!rm -rf result.*

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.8-bin-hadoop2.7.tgz
!pip install findspark

In [ ]:
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar"

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random
import pyspark
import sys
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import math
from pyspark.sql.functions import *
from uuid import uuid1
import time

kafka_topic_name = "hope-pyspark-test"
kafka_bootstrap_servers = 'ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092'

In [31]:
ssc.stop()
sc.stop()

In [5]:
sc = pyspark.SparkContext()
ssc = StreamingContext(sc,1)

kvs = KafkaUtils.createStream(ssc, kafka_bootstrap_servers, 'spark-streaming-consumer', {kafka_topic_name:1}) 
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {"metadata.broker.list": kafka_bootstrap_servers})
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'largest'})

In [6]:
map1 = kvs.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])
reduce = map1.reduceByKey(lambda x,y:x+y)

tf = reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))
map3 = reduce.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
map4 = map3.map(lambda x:(x[0],x[1][2]))
reduce2 = map4.reduceByKey(lambda x,y:x+y)
idf = reduce2.map(lambda x: (x[0], math.log10(337/x[1])))
rdd = tf.join(idf)
rdd = rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1])))
rdd = rdd.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))
rdd.pprint()
# spark = SparkSession(sc)

# rdd.toDF(["DocumentId","Token","TF","IDF","TF-IDF"]).show()
rdd.saveAsTextFiles("result.csv")

ssc.start()
# stream will run for 30 sec
ssc.awaitTerminationOrTimeout(15)

ssc.stop()
sc.stop()

-------------------------------------------
Time: 2021-09-24 13:34:13
-------------------------------------------

-------------------------------------------
Time: 2021-09-24 13:34:14
-------------------------------------------

-------------------------------------------
Time: 2021-09-24 13:34:15
-------------------------------------------

-------------------------------------------
Time: 2021-09-24 13:34:16
-------------------------------------------

-------------------------------------------
Time: 2021-09-24 13:34:17
-------------------------------------------

-------------------------------------------
Time: 2021-09-24 13:34:18
-------------------------------------------

-------------------------------------------
Time: 2021-09-24 13:34:19
-------------------------------------------

-------------------------------------------
Time: 2021-09-24 13:34:20
-------------------------------------------
('0', 'boy', 2, 2.2265999052073573, 4.453199810414715)
('1', 'boy', 1, 2.22659990